In [5]:
import os
import shutil
import cv2

# Set the paths
template_path = 'recc_vlog3/act_g0014.jpg'
target_folder = 'recc_vlog3'
output_folder = 'recc_vlog3/act_g0014'

In [ ]:
def crop_image(image, x1, y1, x2, y2):
    """
    Crops the input image using the provided bounding box coordinates.

    Args:
    image (numpy.ndarray): The input image in OpenCV format.
    x1 (int): The x-coordinate of the top-left corner of the bounding box.
    y1 (int): The y-coordinate of the top-left corner of the bounding box.
    x2 (int): The x-coordinate of the bottom-right corner of the bounding box.
    y2 (int): The y-coordinate of the bottom-right corner of the bounding box.

    Returns:
    numpy.ndarray: The cropped image in OpenCV format.
    """
    # Extract the region of interest from the input image
    cropped_image = image[y1:y2, x1:x2]

    # Return the cropped image
    return cropped_image

def convert_video_to_images(video_file, output_folder, save_every_n_frames):
    # Open the video file
    video = cv2.VideoCapture(video_file)

    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Loop through the frames of the video
    for frame_num in range(total_frames):
        # Read the next frame of the video
        ret, frame = video.read()
        frame_cropped = crop_image(frame,255,130,255+1024,130+768)

        # Check if the frame was successfully read
        if not ret:
            break

        # Check if we should save this frame as an image
        if frame_num % save_every_n_frames == 0:
            # Convert the frame to JPG format
            jpg_frame = cv2.imwrite(f"{output_folder}/frame_{frame_num}.jpg", frame_cropped)

            # Check if the frame was successfully saved as an image
            if not jpg_frame:
                print(f"Failed to save frame {frame_num} as an image.")

    # Release the video file
    video.release()

In [6]:
# Load the template image
template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
template_height, template_width = template.shape[:2]

In [7]:
# Initialize the counters
total_files = 0
matched_files = 0

In [8]:
# Iterate over the files in the target folder
for filename in os.listdir(target_folder):
    if filename.endswith('.jpg'):
        total_files += 1
        
        # Load the target image
        target_path = os.path.join(target_folder, filename)
        target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)
        
        # Perform template matching
        result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # Check if the template is found in the target image
        if max_val >= 0.95:  # Adjust the threshold as needed
            matched_files += 1
            
            # Copy the matched file to the output folder
            output_path = os.path.join(output_folder, filename)
            shutil.copy(target_path, output_path)

            # Display the matched file name and its max_val
            print(f"Matched file: {filename} - Max correlation value: {max_val:.2f}")

# Print the summary report
print(f"Total files processed: {total_files}")
print(f"Matched files: {matched_files}")
print(f"Copied files to output folder: {matched_files}")

Matched file: act_g0014.jpg - Max correlation value: 1.00
Matched file: frame_212.jpg - Max correlation value: 1.00
Matched file: frame_213.jpg - Max correlation value: 1.00
Matched file: frame_282.jpg - Max correlation value: 1.00
Matched file: frame_283.jpg - Max correlation value: 1.00
Matched file: frame_284.jpg - Max correlation value: 1.00
Matched file: frame_285.jpg - Max correlation value: 1.00
Matched file: frame_286.jpg - Max correlation value: 1.00
Matched file: frame_287.jpg - Max correlation value: 1.00
Matched file: frame_288.jpg - Max correlation value: 1.00
Matched file: frame_289.jpg - Max correlation value: 1.00
Matched file: frame_290.jpg - Max correlation value: 1.00
Total files processed: 393
Matched files: 12
Copied files to output folder: 12
